In [1]:
import make_val_prepare_data_aclImdb as imdb_data
from tensorflow.keras.layers import TextVectorization

# 一元语法
# text_vectorization = TextVectorization(
#     max_tokens=10000,
#     output_mode="multi_hot")

# 二元语法
# text_vectorization = TextVectorization(
#     ngrams=2,
#     max_tokens=10000,
#     output_mode="multi_hot"
# )

# 二元语法TF-IDF编码 
def tfidf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
    return term_freq / doc_freq
    
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=10000,
    output_mode="tf_idf",
)

text_only_train_ds = imdb_data.train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)

# use textvectorzation in tf_data channel way
# binary_1gram_train_ds = imdb_data.train_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
# binary_1gram_val_ds = imdb_data.val_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
# binary_1gram_test_ds = imdb_data.test_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)

# use textvectorzation in tf_data channel way
# binary_2gram_train_ds = imdb_data.train_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
# binary_2gram_val_ds = imdb_data.val_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
# binary_2gram_test_ds = imdb_data.test_ds.map(
#     lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)

# # use textvectorzation in tf_data channel way
tfidf_2gram_train_ds = imdb_data.train_ds.map(
    lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
tfidf_2gram_val_ds = imdb_data.val_ds.map(
    lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)
tfidf_2gram_test_ds = imdb_data.test_ds.map(
    lambda x, y: (text_vectorization(x),y),num_parallel_calls =4)



2024-12-21 16:11:00.130920: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 16:11:00.145074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 16:11:00.162510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 16:11:00.165989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 16:11:00.174401: I tensorflow/core/platform/cpu_feature_guar

Found 22500 files belonging to 2 classes.


I0000 00:00:1734768663.154441   41477 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734768663.245799   41477 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734768663.245850   41477 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734768663.249313   41477 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734768663.249358   41477 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Found 2500 files belonging to 1 classes.
Found 25000 files belonging to 2 classes.
train_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
val_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>
test_ds
input shape: (32,)
input shape: <dtype: 'string'>
targets shape: (32,)
targets shape: <dtype: 'int32'>
input[0]: (32,)
targets[0]: <dtype: 'int32'>


2024-12-21 16:12:06.866776: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
import keras
from keras import layers

def get_model(max_token=10000, hidden_dim=16):
    inputs = keras.Input(shape=(max_token,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                 loss="binary_crossentropy",
                 metrics=["accuracy"])
    return model

In [3]:
model = get_model()
model.summary()

# callbacks = [
#     keras.callbacks.ModelCheckpoint("binary_1gram.keras",
#                                     save_best_only=True),
#      keras.callbacks.TensorBoard(log_dir="logs_binary_1grm")
# ]

# # cache data at first epoch,then use cache data in next epochs
# model.fit(binary_1gram_train_ds.cache(),
#           validation_data=binary_1gram_val_ds.cache(),
#           epochs=10,
#           callbacks=callbacks)

# callbacks = [
#     keras.callbacks.ModelCheckpoint("binary_2gram.keras",
#                                     save_best_only=True),
#      keras.callbacks.TensorBoard(log_dir="logs_binary_2grm")
# ]
# # cache data at first epoch,then use cache data in next epochs
# model.fit(binary_2gram_train_ds.cache(),
#           validation_data=binary_2gram_val_ds.cache(),
#           epochs=10,
#           callbacks=callbacks)

callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True),
     keras.callbacks.TensorBoard(log_dir="logs_tfidf_2grm")
]
# cache data at first epoch,then use cache data in next epochs
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks) 



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,033 (625.13 KB)

 Trainable params: 160,033 (625.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


I0000 00:00:1734768731.258799   41547 service.cc:146] XLA service 0x7f49b9abc3a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734768731.259940   41547 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2024-12-21 16:12:11.512832: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-21 16:12:11.917978: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90100


  1/704 ━━━━━━━━━━━━━━━━━━━━ 44:00 4s/step - accuracy: 0.5938 - loss: 4.8759

I0000 00:00:1734768733.036363   41547 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 109ms/step - accuracy: 0.7385 - loss: 0.7479 - val_accuracy: 0.8256 - val_loss: 0.3954
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8654 - loss: 0.3340 - val_accuracy: 0.8876 - val_loss: 0.3313
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8807 - loss: 0.2988 - val_accuracy: 0.8820 - val_loss: 0.2748
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8922 - loss: 0.2745 - val_accuracy: 0.8296 - val_loss: 0.4163
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8931 - loss: 0.2685 - val_accuracy: 0.8084 - val_loss: 0.5322
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8976 - loss: 0.2586 - val_accuracy: 0.8056 - val_loss: 0.4722
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8954 - loss: 0.2634 - val_accuracy: 0.8444 - val_loss: 0.4066
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9008 - loss: 0.2516 - val_accuracy: 0.8528 - v

In [5]:
# model = keras.models.load_model("binary_1gram.keras")
# print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

# model = keras.models.load_model("binary_2gram.keras")
# print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")



782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 92ms/step - accuracy: 0.8804 - loss: 0.3263
Test acc: 0.882
